## Generates dummy data

In [1]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from sample_data import value_sign,day,adequacy_list,choices,last_week_date, last_month_date,test_type, results_list, genotype_list

In [3]:
def week_day(n):
    days_of_the_week=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return days_of_the_week[n%7]

## Generates the numer of tests processed in a given date

In [4]:
### this function defines how busy each day of the week is
def busy_days(n):
    ans=70
    if n==6:
        ans=0
    return ans
    
    
    

In [20]:
### this function defines how busy each month is
def busy_months(n):
    ans=1
    if n ==12:
        ans= 0.8
    if n in [1,2,6,7]:
        ans=1.2
    return ans    
    
    

In [21]:
def number_of_tests(fecha):
    ans=int (busy_days(fecha.weekday())*busy_months(fecha.month)*np.random.uniform(0.95,1.05))
    return ans


## Generate a new row given a data

In [22]:
def make_row(fecha):
    type_of_test=random.choices(test_type[1:],weights=[3,4])[0]
    adequacy=random.choices(adequacy_list,weights=[5,2,1])[0]
    result=random.choices(results_list[1:],weights=[4,1,1,1,1])[0]
    gen_type=random.choice(genotype_list)
    cytology=random.choices(value_sign,weights=[4,6])[0]
    hystology="Not prescribed"
    if cytology=="Positive":
        hystology=random.choice(value_sign)
    
    new_row = pd.DataFrame([{'day':fecha.date(), 'type':type_of_test,'adequacy':adequacy, 'result':result,'mvp':gen_type,'cytology':cytology,'hystology':hystology,"test_quality":"True negative"}])
    return new_row

## Choose the date range 

In [23]:
starting_date = date.today() - timedelta(days = 750)
ending_date=date.today() +timedelta(days = 50)

In [24]:
daterange = pd.date_range(starting_date, ending_date, freq='D')

## Generate test_df

In [25]:
def generate_test_df():
    columns=['day', 'type', 'adequacy', 'result', 'mvp', 'cytology', 'hystology','test_quality']
    test_df = pd.DataFrame(columns=columns)
    daterange = pd.date_range(starting_date, ending_date, freq='D')
    for item in daterange:
        N_of_test=number_of_tests(item)
        for patient in range(N_of_test):
            new_row=make_row(item)
            test_df = pd.concat([test_df, new_row], axis=0, ignore_index=True)
    return test_df

In [26]:
test_df=generate_test_df()

In [168]:
date.today().weekday()

1

In [170]:
test_df["day"][0].weekday()

0

## Generate the initial df


In [27]:
def new_row_initial(fecha,df):
    new_row_in={"day":fecha}
    test_df_fecha=df[test_df["day"]==fecha]
    new_row_in["All test types"]=test_df_fecha.shape[0]
    for tipo in test_type[1:]:
        test_df_fecha_tipo=test_df_fecha[test_df_fecha["type"]==tipo]
        new_row_in[tipo]=test_df_fecha[test_df_fecha["type"]==tipo].shape[0]
    new_row_in["weekday"]=week_day(fecha.weekday())
    return pd.DataFrame([new_row_in])





In [28]:
new_row_initial(date(2022,12,25),test_df)

,day,All test types,Liquid based,Conventional,weekday
0,2022-12-25,0,0,0,Sunday


In [29]:
def make_initial_df(df):
    fechas=df["day"].unique()
    initial_columns=["day","All test types"]+test_type[1:]+["weekday"]
    initial_df=pd.DataFrame(columns=initial_columns)
    for fecha in fechas:
        initial_df=pd.concat([initial_df,new_row_initial(fecha,df)],axis=0, ignore_index=True)
    
    return initial_df

In [30]:
test_df["day"][test_df["day"]==date(2022,12,25)].shape

(0,)

In [31]:
initial_df=make_initial_df(test_df)

In [32]:
initial_df.head(30)

,day,All test types,Liquid based,Conventional,weekday
0,2020-11-30,70,35,35,Monday
1,2020-12-01,57,27,30,Tuesday
2,2020-12-02,56,28,28,Wednesday
3,2020-12-03,53,23,30,Thursday
4,2020-12-04,53,15,38,Friday
5,2020-12-05,57,24,33,Saturday
6,2020-12-07,56,30,26,Monday
7,2020-12-08,57,24,33,Tuesday
8,2020-12-09,56,30,26,Wednesday
9,2020-12-10,56,29,27,Thursday


In [33]:
initial_df.to_csv('test_type_count_T.csv')
test_df.to_csv('test_dataframe_T.csv')